# EyeOn Supermarket data science case study

In [4]:
#pip install --user pyarrow

# Requirement already satisfied: pyarrow in /usr/local/lib/python3.6/dist-packages (0.14.1)
# Requirement already satisfied: numpy>=1.14 in /usr/local/lib/python3.6/dist-packages (from pyarrow) (1.18.5)
# Requirement already satisfied: six>=1.0.0 in /usr/local/lib/python3.6/dist-packages (from pyarrow) (1.15.0)

### Importing packages

In [5]:
import pandas as pd
import sys

### Functions

In [6]:
def f_concat(l_input):

  # Initialize.
  dummy = ""
  n_len = len(l_input)

  # Loop through text elements.
  for i in range(n_len):
    dummy = dummy + l_input[i] + ", "

  # Append last element.
  dummy = dummy + "and " + l_input[n_len-1]

  # Return result.
  return dummy

In [7]:
def f_info(df_input, c_feature, n_top =  10):

  # Testing!
  #df_input  = df_historyPerYear
  #c_feature = "year"
  #n_top     = 3

  # Load package.
  from collections import Counter

  # Bereken frequenties.
  c = Counter(df_input[c_feature])

  # Converteer naar data frame.
  df_output         = pd.DataFrame(list(c.items()))

  # Hernoem kolomnamen.
  df_output.columns = ["level", "n"]

  # Bereken percentage.
  df_output["perc"] = round(100 * df_output["n"] / df_input.shape[0], 1).astype(str) + "%"

  # Sorteer data frame op frequentie.
  df_output         = df_output.sort_values(by = "perc", ascending = False)

  # Reset index en verwijder index kolom die daardoor ontstaat.
  df_output         = df_output.reset_index().drop(columns=['index'])

  # Print header
  print("Frequentietabel voor '" + c_feature + "':", str(df_output["level"].nunique()), "levels.\n")

  # Display tabel zonder index.
  # https://stackoverflow.com/questions/61363712/how-to-print-a-pandas-io-formats-style-styler-object
  #display(df_output.head(n_top).style.hide_index())

  print(df_output.head(n_top))

### Importing data
Importing the parquet files took a bit of fidling. I came across a few URLs, included for reference ([unable-to-read-a-parquet-file]("https://stackoverflow.com/questions/55147424/unable-to-read-a-parquet-file"),   [google-colab-dealing-with-files]("https://neptune.ai/blog/google-colab-dealing-with-files")). Though in the end, I simply connected my Google Drive to Colab and updated the path the to data.

In [17]:
%%time

df_historyPerYear = pd.read_parquet('/Users/home/OneDrive - IWD/Clients/PE/2021 01 - Execute/EyeOn/Data/history-per-year.parquet')
#df_historyPerYear = pd.read_parquet('drive/MyDrive/Colab Notebooks/2021 01 - Execute/EyeOn/Data/history-per-year.parquet')

CPU times: user 9.91 s, sys: 3.86 s, total: 13.8 s
Wall time: 5.41 s


In [18]:
df_historyPerYear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125497040 entries, 0 to 125497039
Data columns (total 8 columns):
 #   Column       Dtype   
---  ------       -----   
 0   id           int64   
 1   store_nbr    uint8   
 2   item_nbr     int64   
 3   unit_sales   float64 
 4   onpromotion  boolean 
 5   day          uint8   
 6   year         category
 7   month        category
dtypes: boolean(1), category(2), float64(1), int64(2), uint8(2)
memory usage: 3.5 GB


In [12]:
# Update formatting of features
df_historyPerYear = df_historyPerYear.astype({"month": int, "year": int})

In [13]:
# A few basic statistics on df_historyPerYear.
print("The 'History per Year' data:\n")
print(f"-> contain {round(df_historyPerYear.shape[0]/1e6, 1)} million observations and {df_historyPerYear.shape[1]} features.\n")
print(f"-> contain {df_historyPerYear.shape[0]} observations and {df_historyPerYear.shape[1]} features.\n")
print(f"-> have feature names: {f_concat(df_historyPerYear.columns)}.\n")
print(f"-> has size of {round(sys.getsizeof(df_historyPerYear)/1024/1024/1024, 1)} GB.")

The 'History per Year' data:

-> contain 125.5 million observations and 8 features.

-> contain 125497040 observations and 8 features.

-> have feature names: id, store_nbr, item_nbr, unit_sales, onpromotion, day, year, month, and month.

-> has size of 5.1 GB.


#### Some Statistics:

In [14]:
df_historyPerYear.head(10)

,id,store_nbr,item_nbr,unit_sales,onpromotion,day,year,month
0,0,25,103665,7.0,<NA>,1,2013,1
1,1,25,105574,1.0,<NA>,1,2013,1
2,2,25,105575,2.0,<NA>,1,2013,1
3,3,25,108079,1.0,<NA>,1,2013,1
4,4,25,108701,1.0,<NA>,1,2013,1
5,5,25,108786,3.0,<NA>,1,2013,1
6,6,25,108797,1.0,<NA>,1,2013,1
7,7,25,108952,1.0,<NA>,1,2013,1
8,8,25,111397,13.0,<NA>,1,2013,1
9,9,25,114790,3.0,<NA>,1,2013,1


In [ ]:
f_info(df_historyPerYear, "store_nbr")

Frequentietabel voor 'store_nbr': 54 levels.

   level        n  perc
0     47  3457407  2.8%
1     44  3513089  2.8%
2     45  3484244  2.8%
3      3  3401264  2.7%
4     46  3353890  2.7%
5     49  3342531  2.7%
6      8  3261184  2.6%
7     48  3236523  2.6%
8      6  3089799  2.5%
9     50  3192566  2.5%


In [15]:
df_historyPerYear.describe()

,id,store_nbr,item_nbr,unit_sales,day,year,month
count,1.254970e+08,1.254970e+08,1.254970e+08,1.254970e+08,1.254970e+08,1.254970e+08,1.254970e+08
mean,6.274852e+07,2.746458e+01,9.727692e+05,8.554865e+00,1.560188e+01,2.015223e+03,6.334971e+00
std,3.622788e+07,1.633051e+01,5.205336e+05,2.360515e+01,8.816411e+00,1.299140e+00,3.392866e+00
min,0.000000e+00,1.000000e+00,9.699500e+04,-1.537200e+04,1.000000e+00,2.013000e+03,1.000000e+00
25%,3.137426e+07,1.200000e+01,5.223830e+05,2.000000e+00,8.000000e+00,2.014000e+03,3.000000e+00
50%,6.274852e+07,2.800000e+01,9.595000e+05,4.000000e+00,1.500000e+01,2.015000e+03,6.000000e+00
75%,9.412278e+07,4.300000e+01,1.354380e+06,9.000000e+00,2.300000e+01,2.016000e+03,9.000000e+00
max,1.254970e+08,5.400000e+01,2.127114e+06,8.944000e+04,3.100000e+01,2.017000e+03,1.200000e+01


In [ ]:
f_info(df_historyPerYear, "year")

Frequentietabel voor 'year': 5 levels.

   level         n   perc
0   2016  35229871  28.1%
1   2015  27864644  22.2%
2   2017  23808261  19.0%
3   2014  22271602  17.7%
4   2013  16322662  13.0%


###Overige data tabellen

In [ ]:
df_history_aggregated = pd.read_parquet('drive/MyDrive/Colab Notebooks/2021 01 - Execute/EyeOn/Data/history_aggregated.parquet')
df_holiday_events     = pd.read_parquet('drive/MyDrive/Colab Notebooks/2021 01 - Execute/EyeOn/Data/holidays_events.parquet')
df_items              = pd.read_parquet('drive/MyDrive/Colab Notebooks/2021 01 - Execute/EyeOn/Data/items.parquet')
df_oil                = pd.read_parquet('drive/MyDrive/Colab Notebooks/2021 01 - Execute/EyeOn/Data/oil.parquet')
df_stores             = pd.read_parquet('drive/MyDrive/Colab Notebooks/2021 01 - Execute/EyeOn/Data/stores.parquet')
df_transactions       = pd.read_parquet('drive/MyDrive/Colab Notebooks/2021 01 - Execute/EyeOn/Data/transactions.parquet')

In [ ]:
df_history_aggregated.head(10)

,onpromotion,item_nbr,week,date,unit_sales_sum,count,family,class,perishable
0,False,764438,201323,2013-06-04 00:00:00+00:00,1925.0,180,GROCERY I,1072,0
1,False,764438,201324,2013-06-11 00:00:00+00:00,1516.0,164,GROCERY I,1072,0
2,False,764438,201325,2013-06-18 00:00:00+00:00,1656.0,176,GROCERY I,1072,0
3,False,764438,201326,2013-06-25 00:00:00+00:00,1610.0,177,GROCERY I,1072,0
4,False,764438,201327,2013-07-02 00:00:00+00:00,1987.0,182,GROCERY I,1072,0
5,False,764438,201328,2013-07-09 00:00:00+00:00,1588.0,169,GROCERY I,1072,0
6,False,764438,201329,2013-07-16 00:00:00+00:00,1688.0,171,GROCERY I,1072,0
7,False,764438,201330,2013-07-23 00:00:00+00:00,1572.0,165,GROCERY I,1072,0
8,False,764438,201331,2013-07-30 00:00:00+00:00,1755.0,176,GROCERY I,1072,0
9,False,764438,201332,2013-08-06 00:00:00+00:00,1780.0,178,GROCERY I,1072,0


In [ ]:
df_holiday_events.head(10)

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False
6,2012-06-23,Holiday,Local,Guaranda,Cantonizacion de Guaranda,False
7,2012-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False
8,2012-06-25,Holiday,Local,Latacunga,Cantonizacion de Latacunga,False
9,2012-06-25,Holiday,Local,Machala,Fundacion de Machala,False


In [ ]:
df_items.head(10)

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1
5,105574,GROCERY I,1045,0
6,105575,GROCERY I,1045,0
7,105576,GROCERY I,1045,0
8,105577,GROCERY I,1045,0
9,105693,GROCERY I,1034,0


In [ ]:
df_oil.head(10)

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21
6,2013-01-09,93.08
7,2013-01-10,93.81
8,2013-01-11,93.60
9,2013-01-14,94.27


In [ ]:
df_stores.head(10)

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4
5,6,Quito,Pichincha,D,13
6,7,Quito,Pichincha,D,8
7,8,Quito,Pichincha,D,8
8,9,Quito,Pichincha,B,6
9,10,Quito,Pichincha,C,15


In [ ]:
df_transactions.head(10)

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
5,2013-01-02,5,1903
6,2013-01-02,6,2143
7,2013-01-02,7,1874
8,2013-01-02,8,3250
9,2013-01-02,9,2940


In [ ]:
df_historyPerYear[df_historyPerYear.item_nbr == 764438].head(5)


,id,store_nbr,item_nbr,unit_sales,onpromotion,day,year,month
1201,1201,1,764438,12.0,None,2,2013,1
2265,2265,2,764438,23.0,None,2,2013,1
3424,3424,3,764438,61.0,None,2,2013,1
4544,4544,4,764438,23.0,None,2,2013,1
5582,5582,5,764438,8.0,None,2,2013,1


In [ ]:
df_history_aggregated[df_history_aggregated.item_nbr == 764438].head(5)

,onpromotion,item_nbr,week,date,unit_sales_sum,count,family,class,perishable
0,False,764438,201323,2013-06-04 00:00:00+00:00,1925.0,180,GROCERY I,1072,0
1,False,764438,201324,2013-06-11 00:00:00+00:00,1516.0,164,GROCERY I,1072,0
2,False,764438,201325,2013-06-18 00:00:00+00:00,1656.0,176,GROCERY I,1072,0
3,False,764438,201326,2013-06-25 00:00:00+00:00,1610.0,177,GROCERY I,1072,0
4,False,764438,201327,2013-07-02 00:00:00+00:00,1987.0,182,GROCERY I,1072,0


In [ ]:
df_items[df_items.item_nbr == 764438].head(3)

,item_nbr,family,class,perishable
941,764438,GROCERY I,1072,0
